<a href="https://colab.research.google.com/github/ahmedmabrouk24/Computer_vision/blob/main/person_Recognetion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tflearn 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 107 kB 14.6 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=8f4cf11f0e8326d812c349ea993cad96e3d8561e239a2c35643eec18b5c67bc8
  Stored in directory: /root/.cache/pip/wheels/65/9b/15/cb1e6b279c14ed897530d15cfd7da8e3df8a947e593f5cfe59
Successfully built tflearn


In [ ]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
import pandas as pd
from tensorflow.keras import layers,models,Sequential
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Dense, Conv2D, Activation, Flatten, Dropout, BatchNormalization, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy
from keras.saving.save import load_model
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import tensorflow as tf
import cv2
import random
import numpy as np
import os
import matplotlib.pyplot as plt
import imageio
import imgaug as ia
import imgaug.augmenters as iaa
from PIL import Image
import shutil
import json


Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
TRAIN_DIR_A = '/content/drive/MyDrive/Computer Vision/Dataset_1/personA/Train'
TRAIN_DIR_B = '/content/drive/MyDrive/Computer Vision/Dataset_1/personB/Train'
TRAIN_DIR_C = '/content/drive/MyDrive/Computer Vision/Dataset_1/personC/Train'
TRAIN_DIR_D = '/content/drive/MyDrive/Computer Vision/Dataset_1/personD/Train'
TRAIN_DIR_E = '/content/drive/MyDrive/Computer Vision/Dataset_1/personE/Train'
TRAIN_DIR=[]
TRAIN_DIR.append(TRAIN_DIR_A)
TRAIN_DIR.append(TRAIN_DIR_B)
TRAIN_DIR.append(TRAIN_DIR_C)
TRAIN_DIR.append(TRAIN_DIR_D)
TRAIN_DIR.append(TRAIN_DIR_E)


Test_DIR_A = '/content/drive/MyDrive/Computer Vision/Dataset_1/personA/Test'
Test_DIR_B = '/content/drive/MyDrive/Computer Vision/Dataset_1/personB/Test'
Test_DIR_C = '/content/drive/MyDrive/Computer Vision/Dataset_1/personC/Test'
Test_DIR_D = '/content/drive/MyDrive/Computer Vision/Dataset_1/personD/Test'
Test_DIR_E = '/content/drive/MyDrive/Computer Vision/Dataset_1/personE/Test'
Test_DIR=[]
Test_DIR.append(Test_DIR_A)
Test_DIR.append(Test_DIR_B)
Test_DIR.append(Test_DIR_C)
Test_DIR.append(Test_DIR_D)
Test_DIR.append(Test_DIR_E)


train_images=[]
test_images=[]


IMG_SIZE = 200
num_classes = 5
MODEL_NAME = 'signature_model1'


In [ ]:
def create_train_data():
    training_data = []

    for path_TRAIN_DIR in tqdm(TRAIN_DIR):
       for img in os.listdir(path_TRAIN_DIR):
        if 'csv' in img:
          continue
        path = os.path.join(path_TRAIN_DIR, img)
        img_data = cv2.imread(path,0) 
        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
        training_data.append([np.array(img_data),ord(img[6])-ord('A') ])

    random.shuffle(training_data)

    return training_data


In [ ]:
column1=[]
def create_test_data():
    test_data = []

    for path_Test_DIR in tqdm(Test_DIR):
       for img in os.listdir(path_Test_DIR):
        if 'csv' in img:
          continue
        column1.append(img)
        path = os.path.join(path_Test_DIR, img)
        img_data = cv2.imread(path,0) 
        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
        test_data.append([np.array(img_data),ord(img[6])-ord('A') ])
       

    #random.shuffle(test_data)
    return test_data


In [ ]:
train_images = create_train_data()
X_train = np.array([i[0] for i in train_images]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_train = np.array([i[1] for i in train_images]).reshape(-1,)

test_images = create_test_data()
X_test = np.array([i[0] for i in test_images]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_test = np.array([i[1] for i in test_images]).reshape(-1,)

In [ ]:

model1 = Sequential([
        layers.Conv2D(filters = 16 ,kernel_size = (3, 3), input_shape = (IMG_SIZE, IMG_SIZE, 1), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size = (2,2)),

        layers.Conv2D(filters = 32 ,kernel_size = (5, 5), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2,2)),
    
        layers.Conv2D(filters = 32 ,kernel_size = (5, 5), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2,2)),
    
        layers.Conv2D(filters = 16 ,kernel_size = (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2,2)),
  
        layers.Flatten(),
        layers.Dense(1024, activation='relu'),
        Dropout(0.5),
        layers.Dense(5, activation = 'softmax')
])

model1.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate = 0.0001),
    loss=SparseCategoricalCrossentropy(),
    metrics=['accuracy'])
if (os.path.exists('/content/drive/MyDrive/person_Recognetion.h5')):
      model1 = load_model('/content/drive/MyDrive/person_Recognetion.h5')
else:
      model1.fit(X_train, y_train, epochs = 25)
      model1.save('/content/drive/MyDrive/person_Recognetion.h5')

  

In [ ]:
print(model1.evaluate(X_test,y_test))

In [ ]:
Y_test=model1.predict(X_test)
print(len(X_test))
prediction=[]
for i in Y_test:
  prediction.append(np.argmax(i))
data={'image_name':column1,'label':prediction[:40]}
print(len(column1),len(prediction))
df_person=pd.DataFrame(data)
print(df_person)
